# Flight Search Project

### dodac: interfejs z filtrem wyszukiwania

In [1]:
import requests
from datetime import datetime
import os
from dotenv import  load_dotenv
import time
from dateutil.relativedelta import relativedelta
from twilio.rest import Client

load_dotenv("D:\Python\devcontainer\.env")

True

In [2]:
TOKEN = os.getenv("SHEETY_TOKEN")
API_KEY_KIWI = os.getenv("API_KEY_KIWI")
ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID")  
ACCOUNT_TOKEN = os.getenv("TWILIO_TOKEN")
PHONE_NUMBER=os.getenv("PHONE_NUMBER")
PHONE_NUMBER_FROM=os.getenv("PHONE_NUMBER_FROM")

In [3]:
# get city id

get_sheet_endpoint = "https://api.sheety.co/c0e1ac8f40667cbfb3ae1345d75a882a/flightDeals/prices"
headers_sheety = {"authorization": f"Bearer {TOKEN}"}

get_city = requests.get(url=get_sheet_endpoint, headers=headers_sheety)

cites = get_city.json()

cites_names = []
rows_id = []

for i in cites['prices']:
    try:
        city = [i]
    except IndexError:
        pass
    else:
        cites_names.append(city[0]["city"])
        rows_id.append(city[0]["id"])

In [4]:
# get IATA code for city

headers_kiwi = {'apikey': API_KEY_KIWI}

cites_iata = []

for name in cites_names:
    kiwi_endpoint = f"https://api.tequila.kiwi.com/locations/query?term={name}&locale=en-US&location_types=airport&limit=10&active_only=true"
    get_info_endpoint = requests.get(url=kiwi_endpoint, headers=headers_kiwi)
    localization_info = get_info_endpoint.json()
    cites_iata.append(localization_info["locations"][0]["city"]["code"])

In [5]:
# upadate IATA code in sheet

num_id = 0

for iata_code in cites_iata:
    new_iata_code = {"price": {
      "iataCode": iata_code,}}
    update_sheet_endpoint = f"https://api.sheety.co/c0e1ac8f40667cbfb3ae1345d75a882a/flightDeals/prices/{rows_id[num_id]}"
    update = requests.put(url=update_sheet_endpoint, headers=headers_sheety, json=new_iata_code)
    num_id += 1
    

In [6]:
def send_sms(my_messege):
    client = Client(ACCOUNT_SID, ACCOUNT_TOKEN)
    client.messages \
                .create(
                        body=my_messege,
                        from_=PHONE_NUMBER_FROM,
                        to=PHONE_NUMBER
                )

In [7]:
# search special offerts

now = datetime.now()
next_six_month = (now + relativedelta(months=6)).strftime(r'%d/%m/%Y').replace("/", r"%2F")
today = (now.strftime(r'%d/%m/%Y')).replace("/", r"%2F")
city_number = 0


for iata_code in cites_iata:

    search_fly_endpoint = f"https://api.tequila.kiwi.com/v2/search?fly_from=WAW&fly_to={iata_code}&date_from={today}&date_to={next_six_month}&nights_in_dst_from=2&nights_in_dst_to=3&max_fly_duration=20&ret_from_diff_city=true&ret_to_diff_city=true&adults=1&selected_cabins=C&mix_with_cabins=M&only_working_days=false&only_weekends=false&partner_market=us&vehicle_type=aircraft&limit=1"

    next_fly = requests.get(url=search_fly_endpoint, headers=headers_kiwi)
    next_fly = next_fly.json()
    try:
        lowest_pice = int(next_fly["data"][0]["price"])
    except Exception:
        city_number += 1
        pass
    else:
        lowest_pice_sheet = int(cites['prices'][city_number]["lowestPrice"])
        fly_details = next_fly["data"][0]
        fly_date = (fly_details["local_departure"]).split("T")
        
        if lowest_pice <= lowest_pice_sheet:
            my_messege = f'Low price alert! Only {lowest_pice}€ to fly from {fly_details["cityFrom"]}-{fly_details["flyFrom"]} to {fly_details["cityTo"]}-{fly_details["flyTo"]}, in {fly_date[0]}.'
            send_sms(my_messege)
        city_number += 1

    time.sleep(3)